In [ ]:
import logging
from typing import Dict, List, Optional, Union, Any

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define environment variables if needed
import os
os.environ.get("OPENAI_API_KEY") or print("Warning: OPENAI_API_KEY not set")
os.environ.get("ANTHROPIC_API_KEY") or print("Warning: ANTHROPIC_API_KEY not set")

# Ember Model Registry Examples

This notebook demonstrates different ways to use models through the Ember model registry.
It covers multiple invocation patterns, from high-level API access to lower-level direct
model usage.

## Setup

First, we'll set up logging and initialize the model registry.

## High-Level API Usage

The recommended way to use Ember models is through the high-level API,
which provides a clean, consistent interface with namespace access.

In [ ]:
# Import the high-level API
from ember.api import models
from ember.api import ModelBuilder, ModelAPI, ModelEnum

# Demonstration of namespace access
try:
    # Using provider namespace
    response = models.openai.gpt4o("What is the capital of France?")
    print(f"Capital of France: {response.data}\n")
    
    # Using model alias
    response = models.gpt4("Tell me a short joke")
    print(f"Joke: {response.data}\n")
except Exception as error:
    logger.exception("Error with namespace access: %s", error)

## Builder Pattern

For more control over model parameters, you can use the builder pattern
to configure the model before invocation.

In [ ]:
try:
    # Create a configured model with the builder pattern
    model = (
        ModelBuilder()
        .temperature(0.7)
        .max_tokens(50)
        .build(ModelEnum.OPENAI_GPT4O)
    )
    
    # Generate response with the configured model
    response = model.generate(prompt="Explain quantum computing in one sentence")
    print(f"Quantum computing: {response.data}")
except Exception as error:
    logger.exception("Error with builder pattern: %s", error)

## Type-Safe Enum Access

For applications that benefit from compile-time checking, you can use
the ModelEnum for type-safe model references.

In [ ]:
try:
    # Using ModelEnum for type safety
    model = ModelAPI.from_enum(ModelEnum.ANTHROPIC_CLAUDE_3_5_SONNET)
    
    response = model.generate(
        prompt="What's your favorite programming language?",
        temperature=0.8
    )
    print(f"Programming language: {response.data}")
except Exception as error:
    logger.exception("Error with enum access: %s", error)

## Direct Registry Access

For advanced usage, you can access the underlying registry directly.

In [ ]:
try:
    # Get the registry
    registry = models.get_registry()
    
    # List available models
    available_models = registry.list_models()
    print(f"Available models: {available_models[:5]}...")
    
    # Get model info for a specific model
    model_info = registry.get_model_info("openai:gpt-4o")
    print(f"Model: openai:gpt-4o")
    print(f"  Name: {model_info.model_name}")
    print(f"  Provider: {model_info.provider.name}")
    print(f"  Input cost: ${model_info.cost.input_cost_per_thousand/1000:.6f} per token")
    print(f"  Output cost: ${model_info.cost.output_cost_per_thousand/1000:.6f} per token")
except Exception as error:
    logger.exception("Error with registry access: %s", error)

## Usage Tracking

Ember provides built-in usage tracking for models. This is useful for
monitoring costs and usage patterns.

In [ ]:
try:
    # Get the usage service
    usage_service = models.get_usage_service()
    
    # Make a few model calls
    models.gpt4("What's the weather like today?")
    models.claude("Tell me about machine learning")
    
    # Get usage statistics
    stats = usage_service.get_usage_stats()
    print(f"Total tokens used: {stats.total_tokens}")
    print(f"Total cost: ${stats.total_cost:.4f}")
    
    # Usage by model
    for model, usage in stats.usage_by_model.items():
        print(f"Model {model}: {usage.total_tokens} tokens, ${usage.cost:.4f}")
except Exception as error:
    logger.exception("Error with usage tracking: %s", error)

## Lower-Level Initialization

If you need more control over the initialization process, you can
initialize the registry and services manually.

In [ ]:
from ember.core.registry.model.initialization import initialize_registry
from ember.core.registry.model.base.services.model_service import ModelService
from ember.core.registry.model.base.services.usage_service import UsageService

try:
    # Initialize registry with custom settings
    registry = initialize_registry(auto_discover=True)
    
    # Create services
    model_service = ModelService(registry=registry)
    usage_service = UsageService(registry=registry)
    
    # Use the service directly
    response = model_service.invoke_model(
        model_id="openai:gpt-4o",
        prompt="Explain the meaning of life",
        temperature=0.7,
        max_tokens=100
    )
    print(f"Meaning of life: {response.data}")
except Exception as error:
    logger.exception("Error with manual initialization: %s", error)

## Manual Model Registration

You can also register models manually if needed.

In [ ]:
from ember.core.registry.model.base.schemas.model_info import ModelInfo
from ember.core.registry.model.base.schemas.provider_info import ProviderInfo
from ember.core.registry.model.base.schemas.cost import ModelCost

try:
    # Create model info
    custom_model = ModelInfo(
        model_id="custom:my-model",
        model_name="My Custom Model",
        cost=ModelCost(
            input_cost_per_thousand=0.001,
            output_cost_per_thousand=0.002
        ),
        provider=ProviderInfo(
            name="CustomProvider",
            default_api_key="your-api-key"
        )
    )
    
    # Register the model
    registry.register_model(custom_model)
    print(f"Registered custom model: {custom_model.model_id}")
    
    # Verify registration
    assert registry.is_registered("custom:my-model")
    print("Verification successful")
except Exception as error:
    logger.exception("Error with manual registration: %s", error)

## Conclusion

This notebook demonstrated various ways to use Ember models, from high-level
API access to lower-level direct usage. The recommended approach for most
use cases is the high-level API with namespace access, but you can choose
the approach that best fits your needs.